# Phase Estimation - 2 and 3 qubits

In [1]:
from pyquil.quil import Program
from pyquil.gates import *
from pyquil import get_qc
import numpy as np
import pandas as pd
import math

In [2]:
def qft_dagger(qc, qubit_list):
    n = len(qubit_list)-1
    for qubit in range(n//2):
        qc += SWAP(qubit_list[qubit], qubit_list[n-qubit-1])
    for j in range(n):
        for m in range(j):
            qc += CPHASE(-math.pi/float(2**(j-m)), qubit_list[j], qubit_list[m])
        qc += H(qubit_list[j])
    return qc

In [3]:
def create_pe_circuit(angle, qubit_list):
    n_qubit = len(qubit_list)-1
    circuit = Program()
    circuit += X(qubit_list[-1])
    for i in range(n_qubit):
        circuit += H(qubit_list[i])
    repetitions = 1
    for counting_qubit in range(n_qubit):
        for i in range(repetitions):
            circuit += CPHASE(angle, qubit_list[counting_qubit], qubit_list[-1])
        repetitions *= 2
    circuit = qft_dagger(circuit, qubit_list)
    ro = circuit.declare('ro', 'BIT', n_qubit)
    for i in range(n_qubit):
        circuit += MEASURE(qubit_list[i], ro[i])
    return circuit

In [14]:
def run_circuit(circuit, qubit_list, qvm):
    n = len(qubit_list) - 1
    exp_array = np.array([])
    for i in range(n):
        exp_array = np.append(exp_array,2**i)
    shots = 10**4
    circuit.wrap_in_numshots_loop(shots)
    #qc = get_qc('4q-qvm')  # You can make any 'nq-qvm' this way for any reasonable 'n'
    qc = get_qc("Aspen-M-1", as_qvm=qvm, execution_timeout=120)
    executable = qc.compile(circuit)
    result = qc.run(executable)
    bitstrings = result.readout_data.get('ro')[:,:n]
    results = bitstrings.dot(exp_array)
    unique_elements, counts_elements = np.unique(results, return_counts=True)
    print(pd.DataFrame({'teta_estimate':unique_elements/2**n, 'freq%':counts_elements/shots * 100}))

## 3 Qubits 

Best qubit in term of fidelity on the Aspen-M-1 Rigetti Processor:

In [5]:
qubit_list = [35,21,36,22] #Aspen-M-1

In [6]:
angle = 2*math.pi/3
circuit = create_pe_circuit(angle, qubit_list)

In [7]:
run_circuit(circuit, qubit_list, qvm = True)

   teta_estimate  freq%
0          0.000   1.66
1          0.125   3.16
2          0.250  17.88
3          0.375  68.39
4          0.500   4.67
5          0.625   1.76
6          0.750   1.37
7          0.875   1.11


In [8]:
run_circuit(circuit, qubit_list, qvm = False)

   teta_estimate  freq%
0          0.000  11.84
1          0.125   8.24
2          0.250  10.00
3          0.375  11.75
4          0.500  12.90
5          0.625  11.13
6          0.750  12.11
7          0.875  22.03


## 5 Qubits 

Best qubit in term of fidelity on the Aspen-M-1 Rigetti Processor:

In [15]:
qubit_list = [35,21,36,22,34,37] #Aspen-M-1

In [16]:
angle = 2*math.pi/3
circuit = create_pe_circuit(angle, qubit_list)

In [11]:
run_circuit(circuit, qubit_list, qvm = True)

    teta_estimate  freq%
0         0.00000   0.11
1         0.03125   0.15
2         0.06250   0.13
3         0.09375   0.11
4         0.12500   0.18
5         0.15625   0.29
6         0.18750   0.47
7         0.21875   0.54
8         0.25000   0.87
9         0.28125   2.73
10        0.31250  17.08
11        0.34375  68.47
12        0.37500   4.57
13        0.40625   1.43
14        0.43750   0.65
15        0.46875   0.45
16        0.50000   0.19
17        0.53125   0.14
18        0.56250   0.17
19        0.59375   0.17
20        0.62500   0.15
21        0.65625   0.13
22        0.68750   0.05
23        0.71875   0.09
24        0.75000   0.14
25        0.78125   0.05
26        0.81250   0.09
27        0.84375   0.07
28        0.87500   0.08
29        0.90625   0.05
30        0.93750   0.07
31        0.96875   0.13


In [12]:
run_circuit(circuit, qubit_list, qvm = False)

    teta_estimate  freq%
0         0.00000   2.12
1         0.03125   2.06
2         0.06250   2.69
3         0.09375   2.40
4         0.12500   2.20
5         0.15625   2.13
6         0.18750   2.61
7         0.21875   2.93
8         0.25000   2.03
9         0.28125   2.29
10        0.31250   3.42
11        0.34375   3.04
12        0.37500   2.23
13        0.40625   3.02
14        0.43750   3.34
15        0.46875   3.75
16        0.50000   2.40
17        0.53125   2.83
18        0.56250   2.74
19        0.59375   2.75
20        0.62500   3.28
21        0.65625   3.58
22        0.68750   3.75
23        0.71875   4.44
24        0.75000   2.72
25        0.78125   2.75
26        0.81250   4.46
27        0.84375   4.19
28        0.87500   3.09
29        0.90625   4.02
30        0.93750   5.26
31        0.96875   5.48
